<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Grid-Search" data-toc-modified-id="Grid-Search-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Grid Search</a></span></li><li><span><a href="#Performances-Profiling" data-toc-modified-id="Performances-Profiling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Performances Profiling</a></span></li></ul></div>

# Intro
Exploratory notebook for running the Chaos Game.

Chaos Game: start from a point inside a polygon and iteratively move to a point between the current position and a randomly chosen vertex of the polygon.
Constraints around distance-factor and vertex-choice can be added to obtain different fractal structures.

In [ ]:
import numpy as np
from math import pi, cos, sin
import cv2
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import json
import tqdm
from collections import namedtuple

%load_ext autoreload
%autoreload 2

from chaos_game import run_chaos_game, _get_polygon_center, get_polygon

from ds_utils.sim_utils import named_configs
from ds_utils.video_utils import generate_video, imageio_generate_video

# Grid Search

In [ ]:
grid_search_params = {
    'segments': [3, 4, 5, 6, 7, 8],
    'nb_iter': [3e5],
    'dist_factor': np.linspace(0., 1., 6, endpoint=False)[1:],
    'vertex_choice_constrain': ['skip_last', 'skip_last_neighbors', ''],
    'width': [1024],
    'mod_gen_image': [1e3],
    'gen_video': [True]
}

out_path = Path.home() / 'Documents/graphics/generative_art_output/chaos_game/1024_grid_search'
out_path.mkdir(exist_ok=True, parents=True)

In [ ]:
configs = list(named_configs(grid_search_params))

for config_idx, config in tqdm.tqdm(enumerate(configs)):
    if config.segments == 3 and config.vertex_choice_constrain == 'skip_last_neighbors':
        continue
    polygon = get_polygon(center=[0.5, 0.5, 0.], radius=.5, segments=config.segments)
    all_points = np.array(run_chaos_game(polygon, nb_iter=config.nb_iter, 
                                         dist_factor=config.dist_factor, 
                                         vertex_choice_constrain=config.vertex_choice_constrain))

    width = config.width
    height = config.width

    img_points = (all_points[:, :2] * (width-1)).astype(np.int16)

    # init black image
    img = np.zeros((height, width, 3), np.uint8)

    images = []
    mod_gen_image = config.mod_gen_image
    for i, p in enumerate(img_points):
        idx = tuple(p)
        img[p[0], p[1]] = img[p[0], p[1]] + [80]*3
        if (i % int(mod_gen_image)) == 0:
            images.append(img.copy())

    if config.gen_video:
        run_name = f'test_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
        with open(out_path / f"{run_name}_settings.txt", "w+", encoding="utf-8") as f:
            json.dump(str(config), f, ensure_ascii=False, indent=4)
        imageio_generate_video(str(out_path / f'{run_name}.mp4'), images, fps=24, format="mp4")


In [ ]:
plt.imshow(images[-1])

# Performances Profiling

In [ ]:
%%prun -s cumulative -l 30 -r
# We profile the cell, sort the report by "cumulative
# time", limit it to 30 lines

config = {
    'segments': 5,
    'nb_iter': 1e4,
    'dist_factor': 0.5,
    'vertex_choice_constrain': 'skip_last',
}
config = namedtuple('Config', config.keys())(**config)


polygon = get_polygon(center=[0.5, 0.5, 0.], radius=.5, segments=config.segments)
all_points = np.array(run_chaos_game(polygon, nb_iter=config.nb_iter, dist_factor=config.dist_factor, 
                                     vertex_choice_constrain=config.vertex_choice_constrain))